In [1]:
import numpy as np
import pandas as pd
from itertools import groupby
from ast import literal_eval

In [2]:
train = pd.read_csv('training.csv')
interest = pd.read_csv('interest_topics.csv')
train['ltiFeatures'] = train['ltiFeatures'].apply(literal_eval)
train['stiFeatures'] = train['stiFeatures'].apply(literal_eval)

In [311]:
aggregation = set([val.split('/')[1] for val in interest['topic_name']])

In [318]:
from collections import defaultdict
dic = defaultdict(list)
user_width_interest = defaultdict(list)
user_aggregate = defaultdict(list)
for i in train['ltiFeatures']:
    temp_dic = defaultdict(list)
    for k, v in i.items():
        value = list(interest.loc[interest['topic_id'] == int(k)]['topic_name'].values)
        if value:
            temp_dic[value[0].split('/')[1]].append(v)
    for a, b in temp_dic.items():
        dic[a].append(np.std(b))
        user_width_interest[a].append(len(b))
        user_aggregate[a].append(sum(b))


In [314]:
stnd_main = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in dic.items()])).fillna(0)
user_width_int = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in user_width_interest.items()])).fillna(0)
user_agg = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in user_aggregate.items()])).fillna(0)

In [3]:
class Tree:
    def __init__(self):
        self.root = '-'
        self.tree = {self.root: {}}

    def add_child(self, heirach):
        current_t, curr = self.tree, self.root
        for i, node in enumerate(heirach):
            if node not in current_t[curr].keys():
                current_t[curr][node] = {}
            current_t = current_t[curr]
            curr = node
    
    def count_leaves(self, subnode):
        def counting(nod, curr):
            if not curr[nod]:
                return 1
            leaves = 0
            for child in curr[nod].keys():
                leaves += counting(child, curr[nod])
            return leaves
        return counting(subnode, self.tree['-'])

In [6]:
from collections import defaultdict
t = Tree()
max_depth = defaultdict(int)
m_width = defaultdict(int)
for i, val in enumerate(interest['topic_name']):
    sp = val.split('/')   
    t.add_child(sp[1:])
    m_width[sp[1]] = max(m_width[sp[1]], len(sp))
#import pprint
#pprint.pprint(t.tree['-']['Arts & Entertainment'])
for i in t.tree['-'].keys():
    max_depth[i] = t.count_leaves(i)
    